<a href="https://colab.research.google.com/github/Baraa710/Premier_League_Predictor/blob/main/data_collect_process/DataScrape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# prompt: code to use firefox webdriver in google colab

!apt-get update
!apt install firefox-esr
!wget https://github.com/mozilla/geckodriver/releases/download/v0.31.0/geckodriver-v0.31.0-linux64.tar.gz
!tar -xzvf geckodriver-v0.31.0-linux64.tar.gz
!mv geckodriver /usr/local/bin
!pip install selenium



Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:7 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,561 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,349 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security/unive

In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import os
import multiprocessing

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [ ]:
df.to_excel('/content/gdrive/MyDrive/APS360_Project/Data/{}.xlsx'.format('2021-2022'), sheet_name='stats_standard')

In [ ]:
ranges = [(n, n+1) for n in range(2017, 2024)]

seasons = ["{}-{}".format(year1,year2) for (year1, year2) in ranges]
stat_types = [('stats','stats_standard'), ('keepers', 'stats_keeper'), ('shooting','stats_shooting'), ('defense','stats_defense'), ('possession','stats_possession')]

In [ ]:
!pip install xlsxwriter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 3.5 MB/s eta 0:00:00


In [ ]:
!pip install openpyxl

In [ ]:

def get_database(season, stat_type, table_id):

  driver.get("https://fbref.com/en/comps/9/{}/{}/{}-Premier-League-Stats".format(season,stat_type, season))
  # Wait for the page to load dynamically
  driver.implicitly_wait(10)

  # Get the page source
  html_data = driver.page_source

  # Parse the HTML data using BeautifulSoup
  soup = BeautifulSoup(html_data, "html.parser")

  # Find the table with id "stats_standard"
  table = soup.find('table', {'id': table_id})
  if table == None:
    print("Could not find table for season {} stat type {}".format(season, stat_type))
    return

  # Extract all rows from the table
  rows = table.find_all('tr')
  col_names = rows[1].find_all('th')
  col_names = [ele.text.strip() for ele in col_names]
  # Create an empty list to store the data
  data = []

  # Loop through the rows and extract the data from each row, skipping the first row
  for row in rows[2:]:
      cols = row.find_all('th') + row.find_all('td')
      cols = [ele.text.strip() for ele in cols]
      if(cols[0].isdigit()!=True):
        continue
      data.append(cols)  # Get rid of empty values

  # Create a DataFrame from the data list
  df = pd.DataFrame(data, columns = col_names)
  filename = '/content/gdrive/MyDrive/APS360_Project/Data/{}.xlsx'.format(season)
  if not os.path.isfile(filename):
    # Create a new file
    with pd.ExcelWriter(filename, engine='xlsxwriter') as writer:
        df.to_excel(writer, sheet_name=table_id)
  else:
    with pd.ExcelWriter(filename, mode='a', if_sheet_exists='new', engine='openpyxl') as writer:
      df.to_excel(writer, sheet_name=table_id)




def process_season(season):
  for i,(stat_type, table_id) in enumerate(stat_types):
    get_database(season, stat_type, table_id)

# Initialize driver
options = webdriver.FirefoxOptions()
options.add_argument("--headless")
driver = webdriver.Firefox(options=options)

num_processes = multiprocessing.cpu_count()
pool = multiprocessing.Pool(processes=num_processes)
pool.map(process_season, seasons)
pool.close()
pool.join()
# Quit the driver
driver.quit()